Основы глубинного обучения, майнор ИАД

Домашнее задание 1. Введение в PyTorch. Полносвязные нейронные сети.
Общая информация

Дата выдачи: 06.10.2021
Мягкий дедлайн: 23:59MSK 25.10.2021
Жесткий дедлайн: 23:59MSK 28.10.2021
Оценивание и штрафы

Максимально допустимая оценка за работу — 10 баллов. За каждый день просрочки снимается 1 балл. Сдавать задание после жёсткого дедлайна сдачи нельзя.
Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).
Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.
О задании

В этом задании вам предстоит предсказывать год выпуска песни по некоторым звуковым признакам: данные. В ячейках ниже находится код для загрузки данных. Обратите внимание, что обучающая и тестовая выборки располагаются в одном файле, поэтому НЕ меняйте ячейку, в которой производится деление данных.

In [1]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import random
from tqdm.notebook import tqdm
import torch.nn.functional as F
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
seed = 228
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [3]:
print(torch.cuda.is_available())
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(dev)

In [4]:
!wget -O data.txt.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip

In [5]:
df = pd.read_csv('data.txt.zip', header=None)
df.head()

Задание 0. (0 баллов, но при невыполнении максимум за все задание — 0 баллов)
Мы будем использовать RMSE как метрику качества. Для самого первого бейзлайна обучите Ridge регрессию из sklearn. Кроме того, посчитайте качество при наилучшем константном прогнозе.

In [6]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

In [7]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

rid = Ridge(alpha=0.1, normalize=True).fit(X_train, y_train)
rmse = np.sqrt(mean_squared_error(y_test,rid.predict(X_test)))

In [8]:
rmse

## Задание 1. (максимум 10 баллов)

Реализуйте обучение и тестирование нейронной сети для предоставленного вам набора данных. Соотношение между полученным значением метрики на тестовой выборке и баллами за задание следующее:

- $\text{RMSE} \le 9.00 $ &mdash; 4 балла
- $\text{RMSE} \le 8.90 $ &mdash; 6 баллов
- $\text{RMSE} \le 8.80 $ &mdash; 8 баллов
- $\text{RMSE} \le 8.75 $ &mdash; 10 баллов

Есть несколько правил, которых вам нужно придерживаться:

- Весь пайплайн обучения должен быть написан на PyTorch. При этом вы можете пользоваться другими библиотеками (`numpy`, `sklearn` и пр.), но только для обработки данных. То есть как угодно трансформировать данные и считать метрики с помощью этих библиотек можно, а импортировать модели из `sklearn` и выбивать с их помощью требуемое качество &mdash; нельзя. Также нельзя пользоваться библиотеками, для которых сам PyTorch является зависимостью.

- Мы никак не ограничиваем ваш выбор архитектуры модели, но скорее всего вам будет достаточно полносвязной нейронной сети.

- Для обучения запрещается использовать какие-либо иные данные, кроме обучающей выборки.

- Ансамблирование моделей запрещено.

### Полезные советы:

- Очень вряд ли, что у вас с первого раза получится выбить качество на 10 баллов, поэтому пробуйте разные архитектуры, оптимизаторы и значения гиперпараметров. В идеале при запуске каждого нового эксперимента вы должны менять что-то одно, чтобы точно знать, как этот фактор влияет на качество.

- Тот факт, что мы занимаемся глубинным обучением, не означает, что стоит забывать про приемы, использующиеся в классическом машинном обучении. Так что обязательно проводите исследовательский анализ данных, отрисовывайте нужные графики и не забывайте про масштабирование и подбор гиперпараметров.

- Вы наверняка столкнетесь с тем, что ваша нейронная сеть будет сильно переобучаться. Для нейросетей существуют специальные методы регуляризации, например, dropout ([статья](https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)) и weight decay ([блогпост](https://towardsdatascience.com/weight-decay-l2-regularization-90a9e17713cd)). Они, разумеется, реализованы в PyTorch. Попробуйте поэкспериментировать с ними.

- Если вы чего-то не знаете, не гнушайтесь гуглить. В интернете очень много полезной информации, туториалов и советов по глубинному обучению в целом и по PyTorch в частности. Но не забывайте, что за скатанный код без ссылки на источник придется ответить по всей строгости!

- Если вы сразу реализуете обучение на GPU, то у вас будет больше времени на эксперименты, так как любые вычисления будут работать быстрее. Google Colab предоставляет несколько GPU-часов (обычно около 8-10) в сутки бесплатно.

- Чтобы отладить код, можете обучаться на небольшой части данных или даже на одном батче. Если лосс на обучающей выборке не падает, то что-то точно идет не так!

- Пользуйтесь утилитами, которые вам предоставляет PyTorch (например, Dataset и Dataloader). Их специально разработали для упрощения разработки пайплайна обучения.

- Скорее всего вы захотите отслеживать прогресс обучения. Для создания прогресс-баров есть удобная библиотека `tqdm`.

- Быть может, вы захотите, чтобы графики рисовались прямо во время обучения. Можете воспользоваться функцией [clear_output](http://ipython.org/ipython-doc/dev/api/generated/IPython.display.html#IPython.display.clear_output), чтобы удалять старый график и рисовать новый на его месте.

**ОБЯЗАТЕЛЬНО** рисуйте графики зависимости лосса/метрики на обучающей и тестовой выборках в зависимости от времени обучения. Если обучение занимает относительно небольшое число эпох, то лучше рисовать зависимость от номера шага обучения, если же эпох больше, то рисуйте зависимость по эпохам. Если проверяющий не увидит такого графика для вашей лучшей модели, то он в праве снизить баллы за задание.

**ВАЖНО!** Ваше решение должно быть воспроизводимым. Если это не так, то проверяющий имеет право снизить баллы за задание. Чтобы зафиксировать random seed, воспользуйтесь функцией из ячейки ниже.

In [9]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
set_random_seed(228)

Проверим данные:

In [10]:
print(df.dtypes.unique())
print(df.isna().values.any())
# пустых ячеек нет, данные все численные 

In [11]:
from sklearn.preprocessing import QuantileTransformer
scaler = QuantileTransformer()

In [12]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715

X_train = torch.tensor(X[:train_size, :])
X_train = scaler.fit_transform(X_train)
y_train = torch.tensor(y[:train_size])
X_test = torch.tensor(X[train_size:, :])
X_test = scaler.fit_transform(X_test)
y_test = torch.tensor(y[train_size:])

In [13]:
y_test

## Класс данных:
Спасибо Кэгл https://www.kaggle.com/matthewmasters/pytorch-nn-starter-baseline-train

In [14]:
class Dataset(torch.utils.data.Dataset):
    """
    Our random dataset
    """
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx, :], self.y[idx]

In [15]:
x = torch.tensor(X_train).to(dev)
y = torch.tensor(y_train).to(dev)

train = Dataset(x, y)

In [16]:
x = torch.tensor(X_test).to(dev)
y = torch.tensor(y_test).to(dev)

test = Dataset(x, y)

Даталоудер:

In [17]:
train_dataloader = torch.utils.data.DataLoader(train, batch_size=64)
test_dataloader = torch.utils.data.DataLoader(test, batch_size=64)

Модель

In [18]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(90, 1500),
            nn.Dropout(0.2),
            nn.BatchNorm1d(1500),
            nn.SiLU(),
            nn.Linear(1500, 500),
            nn.BatchNorm1d(500),
            nn.SiLU(),
            nn.Linear(500, 1)
 
        )
    
    def forward(self, features):
        return self.layers(features.float())

model = Network().to(dev)

Оптимизатор

In [19]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 0.001)

критерий

In [20]:
def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

criterion = RMSELoss

трейн:

In [21]:
def train(model, optimizer, criterion, train_dataloader, test_dataloader):
    '''
    params:
        model - torch.nn.Module to be fitted
        optimizer - model optimizer
        criterion - loss function from torch.nn
        train_dataloader - torch.utils.data.Dataloader with train set
        test_dataloader - torch.utils.data.Dataloader with test set
                      (if you wish to validate during training)
    '''
    for epoch in range(10):
        print('Epoch # {}'.format(epoch))
        train_loss = []
        for x_train, y_train in tqdm(train_dataloader):    # берем батч из трейн лоадера
            y_pred = model(x_train)                        # делаем предсказания
            loss = criterion(y_pred, y_train)
            loss.backward()                                # считаем градиенты обратным проходом
            optimizer.step()                               # обновляем параметры сети
            optimizer.zero_grad()                          # обнуляем посчитанные градиенты параметров
            with torch.no_grad():                          # на валидации запрещаем фреймворку считать градиенты по параметрам # берем батч из вал лоадера
                y_pred = model(x_train)                  # делаем предсказания
                loss = criterion(y_pred, y_train).to('cpu')        # считаем лосс
                train_loss.append(loss.numpy())
    
        model.eval()
        val_loss = []
        
        with torch.no_grad():
            for x_val, y_val in tqdm(test_dataloader):
                y_pred = model(x_val)
                y_val.cpu
                y_pred.cpu
                loss = criterion(y_pred, y_val).to('cpu')
                val_loss.append(loss.numpy())
        print(f"Epoch: {epoch},train_loss: {np.mean(train_loss)}, test_loss: {np.mean(val_loss)}")

In [22]:
train(model, optimizer, criterion, train_dataloader, test_dataloader)

## Задание 2. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Напишите небольшой отчет о том, как вы добились полученного качества: какие средства использовали и какие эксперименты проводили. Подробно расскажите об архитектурах и значениях гиперпараметров, а также какие метрики на тесте они показывали. Чтобы отчет был зачтен, необходимо привести хотя бы 3 эксперимента.

### По порядку
* Я скейлил только трейн из данных, а скейлить тарргеты особо не помогало
* Очееь сильно разнились результаты пока я не зафиксировал сид... да, функцию надо не только инцииализировать, но и выполнить
* Попробовал много оптимизаторов - Adagrad, SGD, Adam, но AdamW сходился быстрее всех и вообще сходился
* пытался сделать РМСЕ и через торч, и через нумпай, но остановился на торче из-за удобства передачи данных в функцию
* по архитектуре сети - опирался на то что в эталонном решение, по словам лектора, было два слоя - пытался эксперементировать с функциями активации от релу и сигмоиды до силу. дропауты не сильно, но улучшали модель
* на табличных данных мне пока что больше нарвистя бустинг:)
* даталоудеры и классы для данных - тоже неочевидная штука, пока не начнешь их делать) на 5к размерах батча ничего вообще не сходилось - снижал до 64 и улучшались метрики
* А ещё кеггл ноутбуки - топ!